In [2]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project='hello-phase3')

투자지표
- 수익성: '영업이익률', '영업이익＜당기＞'
- 성장성: '매출액증가율', '매출액＜전기＞', '매출액＜당기＞'
- 안정성: '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용'

In [3]:
sql_y = """
select stock_code, corp_name, year, account_name, amount
from `hello-phase3.naver_fn_data.investment_index_year`
where account_name in ('영업이익률', '영업이익＜당기＞', '매출액증가율', '매출액＜전기＞', '매출액＜당기＞', '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용')
"""

sql_q = """
select stock_code, corp_name, year, quarter, account_name, amount
from `hello-phase3.naver_fn_data.investment_index_quarter`
where account_name in ('영업이익률', '영업이익＜당기＞', '매출액증가율', '매출액＜전기＞', '매출액＜당기＞', '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용')
"""

df_loaded_y = client.query_and_wait(sql_y).to_dataframe()
df_loaded_q = client.query_and_wait(sql_q).to_dataframe()

df_y = df_loaded_y.drop_duplicates(['stock_code', 'account_name', 'amount'], keep='first').sort_values(['year', 'account_name'])
df_q = df_loaded_q.drop_duplicates(['stock_code', 'account_name', 'amount'], keep='first').sort_values(['year', 'quarter', 'account_name'])

df_y['account_name'] = df_y['account_name'].str.replace('＜당기＞', '')
df_y['account_name'] = df_y['account_name'].str.replace('＜전기＞', '_전기')

df_q['account_name'] = df_q['account_name'].str.replace('＜당기＞', '')
df_q['account_name'] = df_q['account_name'].str.replace('＜전기＞', '_전기')

### 1. [매출] 최근년도 or 최근분기(연환산) 매출 500억 이상

In [4]:
recent_year = 2022
df_1_y = df_y.copy()[(df_y['year'] == recent_year) & (df_y['account_name'] == '매출액')]
df_1_y['result'] = df_1_y['amount'] >= 500

this_year = recent_year + 1
df_1_q = df_q.copy()[(df_q['year'] == this_year) & (df_q['account_name'] == '매출액')]
df_1_q = df_1_q[df_1_q['quarter'] != 4]
df_1_q = df_1_q.groupby('stock_code')['amount'].sum().reset_index()
df_1_q['amount'] = (df_1_q['amount'] / 3) * 4

df_1 = pd.merge(df_1_y, df_1_q, on='stock_code', how='inner')
df_1 = df_1[['stock_code', 'corp_name', 'amount_x', 'amount_y']]
df_1.columns = ['stock_code', 'corp_name', '매출_2022', '매출_2023(연환산)']
df_1['매출_500억이상'] = (df_1['매출_2022'] >= 500) | (df_1['매출_2023(연환산)'] >= 500)
df_1


,stock_code,corp_name,매출_2022,매출_2023(연환산),매출_500억이상
0,042520,한스바이오메드,744.589990,772.812371,True
1,004310,현대약품,1626.900460,1802.576625,True
2,003610,방림,1557.681734,1310.167056,True
3,008870,금비,2349.523928,2527.398582,True
4,093380,풍강,834.225388,970.469491,True
...,...,...,...,...,...
2313,030790,비케이탑스,319.120904,112.417769,False
2314,071950,코아스,978.864707,698.807849,True
2315,078860,아이오케이,189.662243,162.686239,False
2316,008500,일정실업,311.169244,307.429203,False


### 2. 매출액 3개년 플러스 성장
  - 20년 대비 22년 매출성장률 0% 이상
  - 3point: 21년↑, 22년↑, 2point: 21년↓, 22년↑, 1point: 21년↑, 22년↓

In [5]:
df_2 = df_y.copy()[(df_y['year'].isin([2022, 2021, 2020])) & (df_y['account_name'] == '매출액')]
df_2 = df_2.pivot_table(index=['stock_code', 'corp_name'], columns='year', values='amount', aggfunc='first').reset_index()
df_2['매출_3개년up'] = (df_2[2022] - df_2[2020]) / df_2[2020]
df_2['매출_3개년'] = (df_2['매출_3개년up'] > 0)
df_2.columns.name = ''
df_2.columns = ['stock_code', 'corp_name', '매출_2020', '매출_2021', '매출_2022', '매출_3개년up', '매출_3개년']
df_2

,stock_code,corp_name,매출_2020,매출_2021,매출_2022,매출_3개년up,매출_3개년
0,000020,동화약품,2720.753838,2930.181120,3404.263273,0.251221,True
1,000040,KR모터스,1178.343751,1335.104379,1170.587405,-0.006582,False
2,000050,경방,3292.247408,3833.571448,3892.275828,0.182255,True
3,000070,삼양홀딩스,24712.261092,31073.133568,33167.778261,0.342159,True
4,000080,하이트진로,22563.229459,22029.038504,24975.548214,0.106914,True
...,...,...,...,...,...,...,...
2310,452300,캡스톤파트너스,NaN,143.876190,122.164910,NaN,False
2311,453860,에이에스텍,89.505200,146.455980,321.636480,2.593495,True
2312,454910,두산로보틱스,201.640080,369.802230,449.536740,1.229402,True
2313,457550,우진엔텍,263.307200,312.367950,373.643360,0.419040,True


In [6]:
df_2a = df_y.copy()[(df_y['year'].isin([2022, 2021])) & (df_y['account_name'] == '매출액증가율')]
df_2a = df_2a.pivot_table(index=['stock_code', 'corp_name'], columns='year', values='amount', aggfunc='first').reset_index()

pt3_condition = (df_2a[2021] > 0) & (df_2a[2022] > 0)
pt2_condition = (df_2a[2021] <= 0) & (df_2a[2022] > 0)
pt1_condition = (df_2a[2021] > 0) & (df_2a[2022] <= 0)
df_2a['매출_3개년추이'] = np.where(pt3_condition, 3, 
                              np.where(pt2_condition, 2, 
                                       np.where(pt1_condition, 1, 0)
                                       )
                              
                              )
df_2a.columns = ['stock_code', 'corp_name', '매출_2021up', '매출_2022up', '매출_3개년추이']
df_2a

,stock_code,corp_name,매출_2021up,매출_2022up,매출_3개년추이
0,000020,동화약품,7.697,16.179,3
1,000040,KR모터스,13.303,-12.322,1
2,000050,경방,16.442,1.531,3
3,000070,삼양홀딩스,25.740,6.741,3
4,000080,하이트진로,-2.368,13.376,2
...,...,...,...,...,...
2306,452300,캡스톤파트너스,NaN,-15.090,0
2307,453860,에이에스텍,63.628,119.613,3
2308,454910,두산로보틱스,83.397,NaN,0
2309,457550,우진엔텍,18.633,19.616,3


In [7]:
df_2 = pd.merge(df_2, df_2a.drop(columns=['corp_name']), how='left', on='stock_code')
df_2

,stock_code,corp_name,매출_2020,매출_2021,매출_2022,매출_3개년up,매출_3개년,매출_2021up,매출_2022up,매출_3개년추이
0,000020,동화약품,2720.753838,2930.181120,3404.263273,0.251221,True,7.697,16.179,3.0
1,000040,KR모터스,1178.343751,1335.104379,1170.587405,-0.006582,False,13.303,-12.322,1.0
2,000050,경방,3292.247408,3833.571448,3892.275828,0.182255,True,16.442,1.531,3.0
3,000070,삼양홀딩스,24712.261092,31073.133568,33167.778261,0.342159,True,25.740,6.741,3.0
4,000080,하이트진로,22563.229459,22029.038504,24975.548214,0.106914,True,-2.368,13.376,2.0
...,...,...,...,...,...,...,...,...,...,...
2310,452300,캡스톤파트너스,NaN,143.876190,122.164910,NaN,False,NaN,-15.090,0.0
2311,453860,에이에스텍,89.505200,146.455980,321.636480,2.593495,True,63.628,119.613,3.0
2312,454910,두산로보틱스,201.640080,369.802230,449.536740,1.229402,True,83.397,NaN,0.0
2313,457550,우진엔텍,263.307200,312.367950,373.643360,0.419040,True,18.633,19.616,3.0


### 3. 당해년도 누적분기 연환산 매출액 YoY -10% 이상

In [8]:
df_3a = df_y.copy()[(df_y['year'] == 2022) & (df_y['account_name'] == '매출액')]
df_3b = df_q.copy()[(df_q['year'] == 2023) & (df_q['account_name'] == '매출액')]
df_3a

,stock_code,corp_name,year,account_name,amount
18,042520,한스바이오메드,2022,매출액,744.589990
134,004310,현대약품,2022,매출액,1626.900460
238,003610,방림,2022,매출액,1557.681734
348,008870,금비,2022,매출액,2349.523928
458,093380,풍강,2022,매출액,834.225388
...,...,...,...,...,...
250020,030790,비케이탑스,2022,매출액,319.120904
250130,071950,코아스,2022,매출액,978.864707
250240,078860,아이오케이,2022,매출액,189.662243
250350,008500,일정실업,2022,매출액,311.169244


In [9]:
df_3b = df_3b[df_3b['quarter'] != 4]
df_3b = df_3b.pivot_table(index=['stock_code', 'corp_name'], columns='quarter', values='amount')
df_3b = df_3b.dropna(how='any').reset_index()
df_3b[2023] = df_3b[[1,2,3]].sum(axis=1) / 3 * 4
df_3b

quarter,stock_code,corp_name,1,2,3,2023
0,000020,동화약품,994.198138,899.903564,874.574089,3691.567721
1,000040,KR모터스,166.335158,250.909212,190.958385,810.937007
2,000050,경방,915.229471,947.878177,984.879636,3797.316378
3,000070,삼양홀딩스,7952.373546,8316.697280,7994.205820,32351.035528
4,000080,하이트진로,6034.746565,6415.534223,6543.849829,25325.507489
...,...,...,...,...,...,...
2237,450080,에코프로머티,2349.720510,2891.410630,2400.097200,10188.304453
2238,450140,코오롱모빌리티그룹,5385.712125,6115.547192,5995.337635,23328.795935
2239,450520,인스웨이브시스템즈,72.427110,67.686550,67.092650,276.275080
2240,452260,한화갤러리아,420.461570,1270.619513,1200.148805,3854.973184


In [10]:
df_3 = pd.merge(df_3a[['stock_code', 'corp_name', 'amount']], df_3b[['stock_code', 2023]], on='stock_code')
df_3.columns = ['stock_code', 'corp_name', '매출_2022', '매출_2023(연환산)']
df_3['매출_2023up(연환산)'] = (df_3['매출_2023(연환산)'] - df_3['매출_2022']) / df_3['매출_2022']
df_3['매출_당해YoY'] = df_3['매출_2023up(연환산)'] > -0.1
df_3

,stock_code,corp_name,매출_2022,매출_2023(연환산),매출_2023up(연환산),매출_당해YoY
0,042520,한스바이오메드,744.589990,772.812371,0.037903,True
1,004310,현대약품,1626.900460,1802.576625,0.107982,True
2,003610,방림,1557.681734,1310.167056,-0.158899,False
3,008870,금비,2349.523928,2527.398582,0.075707,True
4,093380,풍강,834.225388,970.469491,0.163318,True
...,...,...,...,...,...,...
2237,217620,디딤이앤에프,608.638927,544.668911,-0.105103,False
2238,071950,코아스,978.864707,698.807849,-0.286104,False
2239,078860,아이오케이,189.662243,162.686239,-0.142232,False
2240,008500,일정실업,311.169244,307.429203,-0.012019,True


### 4. 매출 최근분기 QoQ -20% 이상

In [11]:
df_4 = df_3b.copy()[['stock_code', 'corp_name', 2, 3]]
df_4['매출_최근QoQup'] = (df_4[3] - df_4[2]) / df_4[2]
df_4['매출_최근QoQ'] = df_4['매출_최근QoQup'] > -0.2
df_4

quarter,stock_code,corp_name,2,3,매출_최근QoQup,매출_최근QoQ
0,000020,동화약품,899.903564,874.574089,-0.028147,True
1,000040,KR모터스,250.909212,190.958385,-0.238934,False
2,000050,경방,947.878177,984.879636,0.039036,True
3,000070,삼양홀딩스,8316.697280,7994.205820,-0.038776,True
4,000080,하이트진로,6415.534223,6543.849829,0.020001,True
...,...,...,...,...,...,...
2237,450080,에코프로머티,2891.410630,2400.097200,-0.169922,True
2238,450140,코오롱모빌리티그룹,6115.547192,5995.337635,-0.019656,True
2239,450520,인스웨이브시스템즈,67.686550,67.092650,-0.008774,True
2240,452260,한화갤러리아,1270.619513,1200.148805,-0.055462,True


### 매출기준 최종 정리

In [12]:
['stock_code', 'code_name', '매출_2020', '매출_2021', '매출_2021up', '매출_2022', '매출_2022up', '매출_2023(연환산)', '매출_2023up(연환산)', '매출_500억이상', '매출_3개년', '매출_3개년추이', '매출_당해YoY', '매출_최근QoQ']

['stock_code',
 'code_name',
 '매출_2020',
 '매출_2021',
 '매출_2021up',
 '매출_2022',
 '매출_2022up',
 '매출_2023(연환산)',
 '매출_2023up(연환산)',
 '매출_500억이상',
 '매출_3개년',
 '매출_3개년추이',
 '매출_당해YoY',
 '매출_최근QoQ']

In [13]:
# df_1[['stock_code', 'corp_name', '매출_500억이상']]
# df_2.drop(columns=['corp_name'])

df_rev = pd.merge(df_1[['stock_code', 'corp_name', '매출_500억이상']], df_2.drop(columns=['corp_name']), how='left', on='stock_code')
df_rev = pd.merge(df_rev, df_3.drop(columns=['corp_name', '매출_2022']), how='left', on='stock_code')
df_rev = pd.merge(df_rev, df_4.drop(columns=['corp_name']), how='left', on='stock_code')
df_rev = df_rev[['stock_code', 'corp_name', '매출_2020', '매출_2021', '매출_2021up', '매출_2022', '매출_2022up', '매출_2023(연환산)', '매출_2023up(연환산)', '매출_500억이상', '매출_3개년', '매출_3개년추이', '매출_당해YoY', '매출_최근QoQ']]

In [14]:
df_rev

,stock_code,corp_name,매출_2020,매출_2021,매출_2021up,매출_2022,매출_2022up,매출_2023(연환산),매출_2023up(연환산),매출_500억이상,매출_3개년,매출_3개년추이,매출_당해YoY,매출_최근QoQ
0,042520,한스바이오메드,800.984565,590.980328,-26.218,744.589990,25.992,772.812371,0.037903,True,False,2.0,True,True
1,004310,현대약품,1329.655116,1398.027531,5.142,1626.900460,16.371,1802.576625,0.107982,True,True,3.0,True,True
2,003610,방림,1165.469566,1309.731601,12.378,1557.681734,18.931,1310.167056,-0.158899,True,True,3.0,False,True
3,008870,금비,2192.583954,2285.811744,4.252,2349.523928,2.787,2527.398582,0.075707,True,True,3.0,True,True
4,093380,풍강,639.127800,768.051490,20.172,834.225388,8.616,970.469491,0.163318,True,True,3.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2319,030790,비케이탑스,98.578518,247.978846,151.555,319.120904,28.689,NaN,NaN,False,True,3.0,NaN,NaN
2320,071950,코아스,982.229371,1107.697858,12.774,978.864707,-11.631,698.807849,-0.286104,True,False,1.0,False,True
2321,078860,아이오케이,209.681421,213.912185,2.018,189.662243,-11.336,162.686239,-0.142232,False,False,1.0,False,True
2322,008500,일정실업,313.303300,305.231260,-2.576,311.169244,1.945,307.429203,-0.012019,False,False,2.0,True,True


In [15]:
df_rev['매출_3개년'] = df_rev['매출_3개년'].fillna(False)
df_rev['매출_당해YoY'] = df_rev['매출_당해YoY'].fillna(False)
df_rev['매출_최근QoQ'] = df_rev['매출_최근QoQ'].fillna(False)
df_rev.query('(매출_500억이상 == True) and (매출_3개년 == True) and (매출_당해YoY == True) and (매출_최근QoQ == True) and (매출_3개년추이 == 3)')

,stock_code,corp_name,매출_2020,매출_2021,매출_2021up,매출_2022,매출_2022up,매출_2023(연환산),매출_2023up(연환산),매출_500억이상,매출_3개년,매출_3개년추이,매출_당해YoY,매출_최근QoQ
1,004310,현대약품,1329.655116,1398.027531,5.142,1626.900460,16.371,1802.576625,0.107982,True,True,3.0,True,True
3,008870,금비,2192.583954,2285.811744,4.252,2349.523928,2.787,2527.398582,0.075707,True,True,3.0,True,True
4,093380,풍강,639.127800,768.051490,20.172,834.225388,8.616,970.469491,0.163318,True,True,3.0,True,True
13,092440,기신정기,1246.495495,1284.596664,3.057,1313.298925,2.234,1415.555853,0.077863,True,True,3.0,True,True
19,067010,이씨에스,703.980303,747.707927,6.211,802.055031,7.268,939.857772,0.171812,True,True,3.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2241,020400,대동금속,1111.199434,1342.761620,20.839,1469.307568,9.424,1388.903271,-0.054723,True,True,3.0,True,True
2242,053060,세동,1193.707900,1284.322500,7.591,1560.513282,21.505,1714.932996,0.098954,True,True,3.0,True,True
2244,025890,한국주강,338.858451,453.096500,33.713,543.711024,19.999,501.378768,-0.077858,True,True,3.0,True,True
2246,025870,신라에스지,627.442766,712.986765,13.634,734.219343,2.978,874.821248,0.191499,True,True,3.0,True,True
